In [1]:
import numpy as np
from numpy import linalg as la
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale

from data_cube import DataCube
from ssm import SSM
from similarity_network_fusion import SNF, cumulated_euc_ts

In [2]:
from tslearn.metrics import dtw, dtw_path

In [21]:
dc = DataCube(
    subjects="all",
    gestures=["3", "4", "5", "6"],
    channels=["2", "4", "6", "8"],
    data_grp="parsed"
)
dc.load_data()
dc.rms_smooth(100, 50)
dc.normalize_modalities(smooth=True)

---

In [6]:
def pass_antagonist_channels(gnum, array):
    """
    pass antagonist only channels for each gesture
    gnum - gesture number (i.e. 1, 2, 3, or 4)
    array - entire array of all channels and time index
    """
    antgnsts = {"3":[0, 2, 3], # channels 4 & 6; 0 is tidx
                "4":[0, 1, 4], # channels 2 & 8; 0 is tidx
                "5":[0, 3, 4], # channels 6 & 8; 0 is tidx
                "6":[0, 1, 2]} # channels 4 & 2; 0 is tidx
    antagonist_array = np.c_[array[:, antgnsts[gnum][0]],
                             array[:, antgnsts[gnum][1]],
                             array[:, antgnsts[gnum][2]]]
    return antagonist_array

---

In [22]:
dict_antag = {}
for s, gdict in dc.data_set_smooth.items():
    print(s)
    dict_antag[s] = {}
    for g, a in gdict.items():
        snf = SNF(pass_antagonist_channels(g[0], a), k=0.5, metric=cumulated_euc_ts)
        # calculate graph weights to find knn
        snf.calc_weights()
        snf.normalize_weights()
        # generate and normalize knn graphs
        snf.calc_knn_weights()
        snf.normalize_knn_weights()
        # fuse graphs
        snf.network_fusion(eta=0.2, iters=50)
        #print(f"subject {s}; gesture {g}")
        #snf.plot_template()
        # save template to dict
        dict_antag[s][g] = snf.fused_similarity_template

30
06
09
03
05
21
17
01
16
26
32
04
08
27
14
23
25
18
29
15
28
12
19
07
20
31
13
36
24
33
11
35
22
34
10
02


---
### PCA on SNF Matrices

In [ ]:
decomp_ant = {}

for s, gdict in dict_antag.items():
    decomp_ant[s] = {}
    for g, a in gdict.items():
        #print(f"subject {s}; gesture {g}; avg {a.mean()}; sd {a.std()}")
        evals, evecs = la.eig(a)
        evals = evals.real
        sort_idx = np.argsort(-evals)
        evals = evals[sort_idx]
        evecs = evecs[:, sort_idx]
        # calc percent of variance explained
        #print(f"subject {s}; gesture {g}; PoV: {evals[0].real / evals.real.sum()}")
        # do PCA
        res = a @ evecs[:, 0]
        res = scale(res.real)
        decomp_ant[s][g] = res
        # plot results
        plt.subplot(211)
        plt.plot(dc.data_set_smooth[s][g][:,0], res)
        plt.subplot(212)
        plt.plot(dc.data_set_smooth[s][g][:,0], dc.data_set_smooth[s][g][:,1])
        plt.plot(dc.data_set_smooth[s][g][:,0], dc.data_set_smooth[s][g][:,2])
        plt.plot(dc.data_set_smooth[s][g][:,0], dc.data_set_smooth[s][g][:,3])
        plt.plot(dc.data_set_smooth[s][g][:,0], dc.data_set_smooth[s][g][:,4])
        plt.show()

---
### Compare PDs between Lower Star Filtered SNF Image

In [8]:
from scipy.ndimage import gaussian_filter
import persim
from persim import plot_diagrams, PersImage, bottleneck
from ripser import lower_star_img

In [23]:
subj_lab = []
gest_lab = []
arrays = []

for s, gdict in dict_antag.items():
    for g, a in gdict.items():
        subj_lab.append(s)
        gest_lab.append(int(g[0]))
        arrays.append(a)

In [24]:
# calculate bottleneck distance between all pds and make predictions
predicts = []
for n, g1 in enumerate(arrays):
    print(n)
    gaus_g1 = gaussian_filter(g1, sigma=1)
    dgm1 = lower_star_img(gaus_g1)
    g1_bottlenecks = []
    for m, g2 in enumerate(arrays):
        gaus_g2 = gaussian_filter(g2, sigma=1)
        dgm2 = lower_star_img(gaus_g2)
        distance_bottleneck, (matching, D) = bottleneck(dgm1, dgm2, matching=True)
        g1_bottlenecks.append(distance_bottleneck)
    g1_bottlenecks = np.array(g1_bottlenecks)
    pred_idx = np.argsort(g1_bottlenecks)[1] # index of 2nd closest array by dtw; 1st closest is self
    predicts.append(gest_lab[pred_idx])

acc = (sum(np.array(gest_lab) == np.array(predicts)) / len(gest_lab)) * 100

print(f"accuracy: {acc}%")

0


/home/svois/anaconda3/lib/python3.6/site-packages/ripser/ripser.py:342: RuntimeWarning: invalid value encountered in maximum
  thisD = np.maximum(thisD, tD)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


---
### Compare DTW Measure between PCA SNF Matrices

In [ ]:
# organize vectors by gesture
gest_dict = {"3":[], "4":[], "5":[], "6":[]}

for s, gdict in decomp_ant.items():
    for g, a in gdict.items():
        if g[0] not in ["3", "4", "5", "6"]: continue
        gest_dict[g[0]].append(a)

In [ ]:
 comp_dict = {"3":{"3":[], "4":[], "5":[], "6":[]},
             "4":{"3":[], "4":[], "5":[], "6":[]},
             "5":{"3":[], "4":[], "5":[], "6":[]},
             "6":{"3":[], "4":[], "5":[], "6":[]}}

for g1 in ["3", "4", "5", "6"]:
    for g2 in ["3", "4", "5", "6"]:
        for i in range(144):
            for j in range(144):
                if i == j and g1 == g2: continue
                comp_dict[g1][g2].append(dtw(gest_dict[g1][i], gest_dict[g2][j]))

In [ ]:
for g1 in ["3", "4", "5", "6"]:
    print(f"gesture {g1} dtw similarities w/ other gestures:")
    for g2 in ["3", "4", "5", "6"]:
        print(f"avg similarity vs gesture {g2}: {np.mean(comp_dict[g1][g2])}")

---
### Compare Cumulated Distance between PCA SNF Matrices

In [ ]:
subj_lab = []
gest_lab = []
arrays = []

for s, gdict in decomp_ant.items():
    for g, a in gdict.items():
        subj_lab.append(s)
        gest_lab.append(int(g[0]))
        arrays.append(a)

# calculate bottleneck distance between all pds and make predictions
predicts = []
for n, g1 in enumerate(arrays):
    g1_dists = []
    for m, g2 in enumerate(arrays):
        dist = cumulated_euc_ts(g1, g2)
        g1_dists.append(dist)
    g1_dists = np.array(g1_dists)
    pred_idx = np.argsort(g1_dists)[1] # index of 2nd closest array by dtw; 1st closest is self
    predicts.append(gest_lab[pred_idx])

acc = (sum(np.array(gest_lab) == np.array(predicts)) / len(gest_lab)) * 100

print(f"accuracy: {acc}%")

---
### Compare PDs of Time Series Sublevel Sets after PCA

In [ ]:
from ripser import ripser, Rips
from persim import plot_diagrams, PersImage, bottleneck
from TDA_helper_fcns import sublevel_set_time_series_dist

In [ ]:
px = 20
sd = 1

In [ ]:
subj_lab = []
gest_lab = []
arrays = []

for s, gdict in decomp_ant.items():
    for g, a in gdict.items():
        subj_lab.append(s)
        gest_lab.append(int(g[0]))
        arrays.append(a)

# calculate bottleneck distance between all pds and make predictions
predicts = []
for n, g1 in enumerate(arrays):
    rips = Rips(maxdim=0, verbose=False) # initialize rips complex
    sls1 = sublevel_set_time_series_dist(g1)
    dgm1 = rips.fit_transform(sls1, distance_matrix=True)[0]
    g1_bottlenecks = []
    for m, g2 in enumerate(arrays):
        sls2 = sublevel_set_time_series_dist(g2)
        dgm2 = rips.fit_transform(sls2, distance_matrix=True)[0]
        distance_bottleneck, (matching, D) = bottleneck(dgm1, dgm2, matching=True)
        g1_bottlenecks.append(distance_bottleneck)
    g1_bottlenecks = np.array(g1_bottlenecks)
    pred_idx = np.argsort(g1_bottlenecks)[1] # index of 2nd closest array by dtw; 1st closest is self
    predicts.append(gest_lab[pred_idx])

acc = (sum(np.array(gest_lab) == np.array(predicts)) / len(gest_lab)) * 100

print(f"accuracy: {acc}%")

---
### SW1Pers

In [ ]:
t1 = gest_dict["3"][0]
t2 = gest_dict["4"][1]
t3 = gest_dict["5"][20]
t4 = gest_dict["6"][1]

In [ ]:
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D

from sklearn.decomposition import PCA

import scipy.interpolate as interp

from ripser import ripser
from persim import plot_diagrams

In [ ]:
def getSlidingWindow(x, dim, Tau, dT):
    N = len(x)
    NWindows = int(np.floor((N-dim*Tau)/dT)) # The number of windows
    if NWindows <= 0:
        print("Error: Tau too large for signal extent")
        return np.zeros((3, dim))
    X = np.zeros((NWindows, dim)) # Create a 2D array which will store all windows
    idx = np.arange(N)
    for i in range(NWindows):
        # Figure out the indices of the samples in this window
        idxx = dT*i + Tau*np.arange(dim) 
        start = int(np.floor(idxx[0]))
        end = int(np.ceil(idxx[-1]))+2
        if end >= len(x):
            X = X[0:i, :]
            break
        # Do spline interpolation to fill in this window, and place
        # it in the resulting array
        X[i, :] = interp.spline(idx[start:end+1], x[start:end+1], idxx)
    return X

In [ ]:
sw = getSlidingWindow(x=t4, dim=20, Tau=1, dT=0.5)
PDs = ripser(sw, maxdim=1)['dgms']

In [ ]:
pca = PCA(n_components = 2)
Y = pca.fit_transform(sw)
eigs = pca.explained_variance_

In [ ]:
gs = gridspec.GridSpec(2, 2)
plt.figure(figsize=(10,10))
ax1 = plt.subplot(gs[0,0])
ax1.scatter(Y[:, 0], Y[:, 1])

ax1 = plt.subplot(gs[0,1])
ax2 = plot_diagrams(PDs)
plt.show()

In [ ]:
gest_dict